In [1]:
cd ~/code/ctl/build/python

/Users/rhl/code/ctl/build/python


In [2]:
%matplotlib inline
import ctl
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.tri import Triangulation
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial import ConvexHull
def sphere_sample(N, dim=4):
    norm = np.random.normal
    normal_deviates = norm(size=(dim, N))
    radius = np.sqrt((normal_deviates**2).sum(axis=0))
    return (normal_deviates/radius).T

h = lambda l : [np.square(l[0]) + np.square(l[1])**2 - np.square(l[2])**2 - np.square(l[3])**2, 2*(l[0]*l[3] + l[1]*l[2]), 2*(l[1]*l[3]-l[0]*l[3])]

In [3]:
D = sphere_sample(1000)
f = h(D.T)

In [4]:
R = np.random.rand(100,2)
V = ctl.vr(R, .5, 3)

In [5]:
print V.dimension(), V.size()

3 309802


In [6]:
K = ctl.Simplicial_complex([ctl.Abstract_simplex(s.tolist()) for s in ConvexHull(D).simplices])

In [7]:
ctl.homology(K)

[1L, 0L, 0L, 1L]

In [8]:
np.median(f[0])

0.14958043136149216

In [9]:
print K.size()

28076


In [10]:
(n,c) = ctl.open_star_vtx_cover(K, lambda c : (f[0][c-1] > .19)*1 )

In [11]:
blowup = ctl.make_blowup(K,c)

In [12]:
def ss_col_i(blowup, i): 
    lst = [cell for (cell,data) in blowup if cell.second_cell().dimension() == i]
    sc = ctl.Prod_simplicial_complex()
    bd = ctl.Simplex_boundary()
    for pd in lst:
        sigma = pd.first_cell()
        tau = pd.second_cell()
        for rho in bd(sigma):
            sc.insert_open_cell(ctl.Simplex_product(rho.cell(), tau))
        sc.insert_open_cell(ctl.Simplex_product(sigma, tau))
    return sc

In [ ]:
DU = ss_col_i(blowup, 0)
I = ss_col_i(blowup, 1)

In [ ]:
print DU.size(), DU.dimension(), I.size(), I.dimension()

In [ ]:
ctl.homology(DU)

In [ ]:
ctl.homology(I)

In [ ]:
(X_bti, X_chains) = ctl.homology(KX)
(Y_bti, Y_chains) = ctl.homology(KY)
(I_bti, I_chains) = ctl.homology(KI)
print "H_*(KX)", X_bti
print "H_*(KY)", Y_bti
print "H_*(KI)", I_bti

In [ ]:
def plot_2skel(X, K):
    X -= X.mean(axis=0)
    triangles = np.array([list(s) for (s,d) in K if s.dimension() == 2])
    rad = np.linalg.norm(X, axis=1)
    zen = np.arccos(X[:,-1] / rad)
    azi = np.arctan2(X[:,1], X[:,0])
    tris = Triangulation(zen, azi)
    fig = plt.figure()
    ax  = fig.add_subplot(111, projection='3d')
    ax.plot_trisurf(X[:,0], X[:,1], X[:,2], triangles=triangles, cmap=plt.cm.bone)
    plt.show()

In [ ]:
plot_2skel(D, KX)

In [ ]:
plot_2skel(D, KY)

In [ ]:
plot_2skel(D, KI)

In [ ]:
ctl.homology(KX)

In [ ]:
ctl.homology(KY)

In [ ]:
ctl.homology(KI)

In [ ]:
Kcheck = ctl.Simplicial_complex()
for (s,d) in KX:
    Kcheck.insert_open_cell(s)
for (s,d) in KY:
    Kcheck.insert_open_cell(s)
for (s,d) in KI:
    Kcheck.insert_open_cell(s)
print Kcheck.is_closed()
ctl.homology(Kcheck)